In [1]:
import os
import sys
import glob
import pandas as pd
import shutil


In [2]:
# === Utility ===

def get_next_run_number(base_dir, prefix="hbdf_milk_run_", ext=".txt"):
    pattern = os.path.join(base_dir, f"{prefix}*{ext}")
    existing = glob.glob(pattern)
    nums = []
    for path in existing:
        name = os.path.basename(path)
        try:
            num = int(name[len(prefix):-len(ext)])
            nums.append(num)
        except ValueError:
            continue
    n = 1
    while n in nums:
        n += 1
    return n

def backup_outsim():
    base_dir = os.path.abspath(ROOT_PATH)
    run_num = get_next_run_number(base_dir, OUTSIM_BACKUP_PREFIX, BACKUP_EXT)
    src_path = os.path.join(base_dir, OUTSIM_FILENAME)
    if not os.path.exists(src_path):
        raise FileNotFoundError(f"outsim file not found: {src_path}")

    backup_name = f"{OUTSIM_BACKUP_PREFIX}{run_num}{BACKUP_EXT}"
    backup_path = os.path.join(base_dir, backup_name)
    shutil.copyfile(src_path, backup_path)
    print(f"✔️  Copied '{OUTSIM_FILENAME}' to '{backup_name}'")

def load_excel_block_lines(xlsx_path, sheet_name, row1, row2):
    """
    Returns a list of formatted HBDF data lines for the given Excel block.
    Uses columns 2..54 (0-based) = C..BC, matching your previous script.
    """
    df = pd.read_excel(xlsx_path, sheet_name=sheet_name)
    flow_data = df.iloc[row1:row2, 2:55]  # year in first col of slice; weeks in the rest

    formatted_lines = []
    for _, row in flow_data.iterrows():
        year = f"{int(row.iloc[0]):4d}"
        flow_parts = []
        for val in row.iloc[1:]:
            # Keep your fixed width (9 total, 3 decimals), right-aligned
            formatted_val = f"{val:9.3f}"
            flow_parts.append(formatted_val)
        line = f"{year} " + ''.join(flow_parts)
        formatted_lines.append(line)
    return formatted_lines

def find_data_block_bounds(lines, header_predicate):
    """
    Find [data_start, data_end) for a block after a header line and a 'Year' header row.
    Falls back to 'first 4-digit year line' if no explicit 'Year' line is present.
    """
    # 1) header line
    header_idx = next((i for i, line in enumerate(lines) if header_predicate(line)), None)
    if header_idx is None:
        return None, None, None  # not found

    # 2) a 'Year' line after header
    year_header_idx = next(
        (j for j in range(header_idx + 1, len(lines)) if lines[j].lstrip().startswith("Year")),
        None
    )

    if year_header_idx is not None:
        data_start = year_header_idx + 1
    else:
        # Fallback: first line that starts with 4-digit year
        data_start = next(
            (j for j in range(header_idx + 1, len(lines))
             if lines[j].strip()[:4].isdigit() and len(lines[j].strip().split()[0]) == 4),
            None
        )
        if data_start is None:
            return header_idx, None, None

    # 3) walk to end of year-data block
    data_end = data_start
    while data_end < len(lines):
        tok = lines[data_end].strip().split()
        # stop when not a 4-digit year at column 0
        if not tok or not tok[0].isdigit() or len(tok[0]) != 4:
            break
        data_end += 1

    return header_idx, data_start, data_end

def update_hbdf_sections_in_memory(hbdf_lines, updates):
    """
    updates: list of dicts with keys:
      - name: e.g., "NATCH982"
      - year: e.g., "1981"
      - units: e.g., "CMS"
      - new_lines: list of formatted data lines to splice in
    Returns the updated hbdf_lines.
    """
    lines = hbdf_lines[:]  # copy to avoid in-place surprises
    for u in updates:
        name = u["name"]
        year = u["year"]
        units = u["units"]
        new_lines = u["new_lines"]

        def header_predicate(line):
            # Match e.g., " NATCH982 1981  35 CMS" -> startswith(" NATCH982") and contains "1981" and "CMS"
            return line.startswith(f" {name}") and (year in line) and (units in line)

        header_idx, data_start, data_end = find_data_block_bounds(lines, header_predicate)

        if header_idx is None:
            raise SystemExit(f"ERROR: Header for '{name} {year} ... {units}' not found.")

        if data_start is None or data_end is None:
            raise SystemExit(f"ERROR: Could not find data block bounds for '{name} {year} ... {units}'.")

        # Replace the data block
        lines = lines[:data_start] + [ln + ("" if ln.endswith("\n") else "\n") for ln in new_lines] + lines[data_end:]
        print(f"✔️  Replaced data block for {name} ({year}, {units})")

    return lines


In [4]:
# === Configuration ===
ROOT_PATH = '/Users/alexandergiberson/Documents/Masters_Work/WRM_TWG_CREDIT_RUNS/Capped_Credit_Run_600cfs'
EXCEL_FILENAME = "Calc_US_TAKE_v6.xlsx"
SHEET_NAME = "Cap"
HBDF_FILENAME = "hbdf.txt"
OUTSIM_FILENAME = "outsim.csv"

# Backups
NEW_HBDF_BASENAME = "New_US_STMR_HBDF.txt"  # not strictly needed now, but kept if you want to dump intermediates
OUTSIM_BACKUP_PREFIX = "outsim_stmr_run_"
HBDF_BACKUP_PREFIX = "hbdf_stmr_run_"
BACKUP_EXT = ".csv"

#    {"name": "CAN STM", "row1": 736, "row2": 771, "year": "1981", "units": "CMS"},
SECTIONS = [
    # Row 1 is the section name row, Row 2 is the second to last year row.
    {"name": "CAN STM", "row1": 736, "row2": 771, "year": "1981", "units": "CMS"},
    # NEW section requested
    {"name": "NATCH982", "row1": 775,  "row2": 810,  "year": "1981", "units": "CMS"},
]

In [16]:
# === Main Execution ===
if __name__ == "__main__":
    base_dir = os.path.abspath(ROOT_PATH)
    excel_path = os.path.join(base_dir, EXCEL_FILENAME)
    hbdf_path = os.path.join(base_dir, HBDF_FILENAME)

    # 1) Prepare new lines for every section from Excel
    updates = []
    for sec in SECTIONS:
        lines_block = load_excel_block_lines(excel_path, SHEET_NAME, sec["row1"], sec["row2"])
        updates.append({
            "name": sec["name"],
            "year": sec["year"],
            "units": sec["units"],
            "new_lines": lines_block
        })

    # 2) Load HBDF and apply all replacements in-memory (single write)
    with open(hbdf_path, "r") as f:
        hbdf_lines = f.readlines()

    updated_hbdf = update_hbdf_sections_in_memory(hbdf_lines, updates)

    # 3) Backup hbdf.txt once, then write updated
    run_num = get_next_run_number(base_dir, HBDF_BACKUP_PREFIX, ext=".txt")
    hbdf_backup_name = f"{HBDF_BACKUP_PREFIX}{run_num}.txt"
    hbdf_backup_path = os.path.join(base_dir, hbdf_backup_name)
    os.rename(hbdf_path, hbdf_backup_path)
    print(f"✔️  Backed up original HBDF to '{hbdf_backup_name}'")

    with open(hbdf_path, "w") as f:
        f.writelines(updated_hbdf)
    print("✔️  Wrote updated contents to 'hbdf.txt'")

    # 4) Backup outsim.csv as before
    backup_outsim()


✔️  Replaced data block for CAN STM (1981, CMS)
✔️  Replaced data block for NATCH982 (1981, CMS)
✔️  Backed up original HBDF to 'hbdf_stmr_run_5.txt'
✔️  Wrote updated contents to 'hbdf.txt'
✔️  Copied 'outsim.csv' to 'outsim_stmr_run_5.csv'
